In [1]:
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X[:3]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [3]:
y[:3]

0    0
1    0
2    0
Name: target, dtype: int64

In [6]:
from hypergbm.estimators import XGBoostEstimator
from hypergbm.pipeline import Pipeline, DataFrameMapper
from hypergbm.sklearn.transformers import MinMaxScaler, StandardScaler
from hypernets.core import OptimizeDirection
from hypernets.core.ops import ModuleChoice, HyperInput
from hypernets.core.search_space import HyperSpace
from hypernets.tabular.column_selector import column_number_exclude_timedelta


def search_space():
    space = HyperSpace()
    with space.as_default():
        input = HyperInput(name='input1')
        scaler_choice = ModuleChoice(
            [
                StandardScaler(name=f'numeric_standard_scaler'),
                MinMaxScaler(name=f'numeric_minmax_scaler')
            ], name=f'numeric_or_scaler'
        )
        num_pipeline = Pipeline([scaler_choice], name='numeric_pipeline', columns=column_number_exclude_timedelta)(input)
        union_pipeline = DataFrameMapper(default=None, input_df=True, df_out=True)([num_pipeline])
        xgb_est = XGBoostEstimator(fit_kwargs={})
        ModuleChoice([xgb_est], name='estimator_options')(union_pipeline)  # Make xgboost as a estimator choice
        space.set_inputs(input)
    return space

In [5]:
from hypergbm import HyperGBM
from hypernets.searchers import MCTSSearcher
rs = MCTSSearcher(search_space, max_node_space=10, optimize_direction=OptimizeDirection.Maximize)
hk = HyperGBM(rs, task='multiclass', reward_metric='accuracy', callbacks=[])
hk.search(X_train, y_train, X_eval=X_test, y_eval=y_test)

17:19:57 I hypernets.m.hyper_model.py 249 - 3 class detected, inferred as a [multiclass classification] task
17:19:57 I hypernets.c.meta_learner.py 22 - Initialize Meta Learner: dataset_id:74e7c134740a0f846f4c5e57fa5e6c93


,trial No.,Previous reward,Best trial,Best reward,Total elapsed,Max trials
0,2,1.0,1,1.0,0.176803,10


#### Current Trial:

,Trial No.,Reward,Elapsed,Space Vector
0,1,1.0,0.122744,[1]
1,2,1.0,0.090189,[0]


Pass classes=[0 1 2], y=22     0
15     0
65     1
11     0
42     0
      ..
71     1
106    2
14     0
92     1
102    2
Name: target, Length: 120, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error


#### Best Trial:

key,value
signature,8a42827ddc672747f568cdd0f9f46c2d
vectors,[1]
0-numeric_or_scaler.hp_or,1


17:19:57 I hypernets.d.in_process_dispatcher.py 119 - Trial 1 done, reward: 1.0, best_trial_no:1, best_reward:1.0



Pass classes=[0 1 2], y=22     0
15     0
65     1
11     0
42     0
      ..
71     1
106    2
14     0
92     1
102    2
Name: target, Length: 120, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error


17:19:57 I hypernets.d.in_process_dispatcher.py 119 - Trial 2 done, reward: 1.0, best_trial_no:1, best_reward:1.0

17:20:08 I hypernets.d.in_process_dispatcher.py 41 - Unable to take valid sample and exceed the retry limit 1000.
